In [2]:
using BenchmarkTools

In [3]:
using Pkg
Pkg.resolve()
Pkg.develop(Pkg.PackageSpec(path = "/Users/kirillvesialou/Desktop/jetReconstruction/JetReconstruction.jl"))
using JetReconstruction
Pkg.status()

  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


Status `~/.julia/environments/v1.11/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0
⌃ [7073ff75] IJulia v1.28.0
  [44e8cb2c] JetReconstruction v0.5.0-dev `~/Desktop/jetReconstruction/JetReconstruction.jl`
  [44cfe95a] Pkg v1.11.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [7]:
function add_ghosts1(event::Vector{PseudoJet}, resolution::Int)
	RapidityStep = 6.28 / resolution
	PhiStep = 10 / resolution
	for i in 0:(resolution - 1)
		for j in 0:(resolution - 1)
			Rap =  (i + (rand() - 0.5)) * RapidityStep
			Phi = (j + (rand() - 0.5)) * PhiStep
            ghost = PseudoJet(pt = 1.0f-45, rap = Rap, phi = Phi, m = 0)
            push!(event, ghost)
        end
    end
	return event
end

add_ghosts1 (generic function with 1 method)

In [7]:
jet1 = PseudoJet(pt = 1.1, rap = 1.1, phi = 1.1)

PseudoJet(px: 0.4989557335681351 py: 0.9803280960675791 pz: 1.469212217136595 E: 1.8353704092044822 cluster_hist_index: 0)

In [8]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

add_ghosts1(event, 100)

10001-element Vector{PseudoJet}:
 PseudoJet(px: 0.8509865563896788 py: -2.8767728239894153 pz: 183.06632985935454 E: 183.1891949001742 cluster_hist_index: 0)
 PseudoJet(px: 1.3998070606540532e-45 py: -6.463419421753941e-47 pz: -3.32450567920983e-47 E: 1.4016927694470713e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.396239209941528e-45 py: 1.1896829300678513e-46 pz: 3.818554174984449e-47 E: 1.4018186479419583e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.3757514245733657e-45 py: 2.663557844377436e-46 pz: -3.0215725253408585e-47 E: 1.4016241922040586e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.3532179091899364e-45 py: 3.639212502268968e-46 pz: -3.585008485412906e-47 E: 1.401756974194578e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.2954184143159795e-45 py: 5.3434868575693514e-46 pz: 7.683077408340429e-48 E: 1.4013195266596242e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.231547228592022e-45 py: 6.685273441426315e-46 pz: -1.9840785460013155e-47 E: 1.4014389187141766e-45 cluster_hist_index: 0)
 Ps

In [9]:
@benchmark add_ghosts1($event, 100) samples = 10000 evals = 10

BenchmarkTools.Trial: 1353 samples with 10 evaluations per sample.
 Range (min … max):  162.079 μs … 89.023 ms  ┊ GC (min … max): 0.00% … 9.75%
 Time  (median):     178.992 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   369.504 μs ±  3.264 ms  ┊ GC (mean ± σ):  5.17% ± 1.02%

                  ▃▆█▇▆█▅▅▆▄▂▂▄ ▂                               
  ▂▁▂▂▂▂▂▃▂▂▂▁▁▂▃▆█████████████▇██▆▆▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▂ ▄
  162 μs          Histogram: frequency by time          205 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
function add_ghosts2(event::Vector{PseudoJet}, resolution::Int)
	RapidityStep = 6.28 / resolution
	PhiStep = 10 / resolution
    rap_v = collect(0:(resolution - 1))
    rap_rand = rand(resolution) .- 0.5
    rap_v .+= rap_rand
    rap_v .* RapidityStep
    phi_v = collect(0:(resolution - 1))
    phi_rand = rand(resolution) .- 0.5
    phi_v .+= phi_rand
    phi_v .* PhiStep
    sizehint!(event, length(event) + resolution)
    @inbounds for i in 1:resolution
        push!(event, PseudoJet(pt = 0.0001, rap = rap_v[i], phi = phi_v[i]))
    end
	return event
end

add_ghosts2 (generic function with 1 method)

In [8]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

add_ghosts2(event, 100)

InexactError: InexactError: Int64(-0.2878990531546266)

In [9]:
@benchmark add_ghosts2($event, 100) samples = 10000 evals = 10

InexactError: InexactError: Int64(0.013197735002202338)

In [10]:
function add_ghosts3(event::Vector{PseudoJet}, resolution::Int)
	RapidityStep = 6.28 / resolution
    PhiStep = 10 / resolution

    rap_v = (0:(resolution - 1)) .+ (rand(resolution) .- 0.5)
    rap_v .*= RapidityStep

    phi_v = (0:(resolution - 1)) .+ (rand(resolution) .- 0.5)
    phi_v .*= PhiStep
    
    sizehint!(event, length(event) + resolution)
    @inbounds for i in 1:resolution
        push!(event, PseudoJet(pt = 0.0001, rap = rap_v[i], phi = phi_v[i]))
    end
	return event
end

add_ghosts3 (generic function with 1 method)

In [11]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

add_ghosts3(event, 100)

101-element Vector{PseudoJet}:
 PseudoJet(px: 0.8509865563896788 py: -2.8767728239894153 pz: 183.06632985935454 E: 183.1891949001742 cluster_hist_index: 0)
 PseudoJet(px: 9.999536803516916e-5 py: 9.624819536341598e-7 pz: -2.6836788414564644e-6 E: 0.00010003600417911584 cluster_hist_index: 0)
 PseudoJet(px: 9.987179113487245e-5 py: 5.06214732227475e-6 pz: 6.6855591479408e-6 E: 0.00010022323433775532 cluster_hist_index: 0)
 PseudoJet(px: 9.755793531976915e-5 py: 2.1964727545402008e-5 pz: 9.873187701618773e-6 E: 0.00010048621714141395 cluster_hist_index: 0)
 PseudoJet(px: 9.675673064481483e-5 py: 2.5261335572901817e-5 pz: 1.652499567997909e-5 E: 0.00010135618127289193 cluster_hist_index: 0)
 PseudoJet(px: 9.065455046516269e-5 py: 4.2210809989376824e-5 pz: 2.3127148669855944e-5 E: 0.00010263949047806901 cluster_hist_index: 0)
 PseudoJet(px: 8.690141022058653e-5 py: 4.947873181148988e-5 pz: 2.9911109367919423e-5 E: 0.00010437755727942495 cluster_hist_index: 0)
 PseudoJet(px: 8.2660644556356

In [12]:
@benchmark add_ghosts3($event, 100) samples = 10000 evals = 10

BenchmarkTools.Trial: 460 samples with 10 evaluations per sample.
 Range (min … max):    4.388 μs …   2.583 ms  ┊ GC (min … max):  0.00% … 29.02%
 Time  (median):     959.567 μs               ┊ GC (median):    28.37%
 Time  (mean ± σ):     1.091 ms ± 649.294 μs  ┊ GC (mean ± σ):  29.08% ±  9.45%

   ▂              ▁▂█    ▃▇▁▄▁ ▅▂             ▁▁           ▁     
  ▆█▃▃▄▅▅▆▇▅▇▅█▄▇████▅▆▃▅█████▆██▅▃▃▁▁▁▃▃▁▃▁▁▄██▆█▆█▅█▅▅▅▅▅█▅▄▄ ▄
  4.39 μs          Histogram: frequency by time         2.31 ms <

 Memory estimate: 105.01 KiB, allocs estimate: 10.

In [13]:
function add_ghosts4!(event::Vector{PseudoJet}, resolution::Int)
    RapidityStep = 6.28 / resolution
    PhiStep = 10 / resolution

    total_new = resolution * resolution
    sizehint!(event, length(event) + total_new)

    @inbounds for i in 0:(resolution - 1)
        for j in 0:(resolution - 1)
            push!(event, PseudoJet(
                0.00001,
                (i + (rand() - 0.5)) * RapidityStep,
                (j + (rand() - 0.5)) * PhiStep,
                0.0
            ))
        end
    end

    return event
end

add_ghosts4! (generic function with 1 method)

In [14]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

add_ghosts4!(event, 100)

10001-element Vector{PseudoJet}:
 PseudoJet(px: 0.8509865563896788 py: -2.8767728239894153 pz: 183.06632985935454 E: 183.1891949001742 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: -0.00048106521372813564 pz: 0.011130871500727469 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: 0.026737305008966573 pz: 0.051688719162421404 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: -0.006494591917994295 pz: 0.16090029803116956 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: 0.02440852198889126 pz: 0.2878216279020253 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: 0.02295844444693159 pz: 0.44288242170648434 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: -0.02477799607685918 pz: 0.5406884314556207 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: -0.027465337551530678 pz: 0.5733188651094654 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: -0.01810239849809993 pz: 0.7427644067977317 E: 0.0 cluster_hist_index: 0)
 PseudoJet(px: 1.0e-5 py: -0.0229

In [ ]:
@benchmark add_ghosts4!($event, 100) samples = 10000 evals = 10

In [ ]:
function add_ghosts_fast!(event::Vector{PseudoJet}, resolution::Int)
    n_ghosts = resolution * resolution
    n_original = length(event)
    
    # Single resize
    resize!(event, n_original + n_ghosts)
    
    RapidityStep = Float32(6.28 / resolution)
    PhiStep = Float32(10.0 / resolution)
    
    # For resolution=100, pre-generating 20k random numbers is fast
    rand_vals = rand(Float32, 2 * n_ghosts) .- 0.5f0
    
    # Unroll the pattern computation
    @inbounds for k in 0:(n_ghosts - 1)
        i = k ÷ resolution
        j = k % resolution
        rand_idx = 2k + 1
        
        Rap = (i + rand_vals[rand_idx]) * RapidityStep
        Phi = (j + rand_vals[rand_idx + 1]) * PhiStep
        
        event[n_original + k + 1] = PseudoJet(0, Rap, Phi, 0)
    end
    
    nothing
end

In [16]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

add_ghosts_fast!(event, 100)

In [ ]:
@benchmark add_ghosts4!($event, 100) samples = 10000 evals = 10

In [1]:
struct GhostGenerator
    resolution::Int
    n_ghosts::Int
    RapidityStep::Float32
    PhiStep::Float32
    
    function GhostGenerator(resolution::Int)
        new(resolution, 
            resolution * resolution,
            Float32(6.28 / resolution),
            Float32(10.0 / resolution))
    end
end

function add_ghosts_cached!(event::Vector{PseudoJet}, gen::GhostGenerator)
    n_original = length(event)
    resize!(event, n_original + gen.n_ghosts)
    
    # Generate random values
    rand_vals = rand(Float32, 2 * gen.n_ghosts) .- 0.5f0
    
    @inbounds for k in 0:(gen.n_ghosts - 1)
        i = k ÷ gen.resolution
        j = k % gen.resolution
        rand_idx = 2k + 1
        
        Rap = (i + rand_vals[rand_idx]) * gen.RapidityStep
        Phi = (j + rand_vals[rand_idx + 1]) * gen.PhiStep
        
        event[n_original + k + 1] = PseudoJet(pt = 0, rap = Rap, phi = Phi, m = 0)
    end
    
    nothing
end

UndefVarError: UndefVarError: `PseudoJet` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

gen= GhostGenerator(100)
add_ghosts_cached!(event, gen)

In [17]:
@benchmark add_ghosts_cached!($event, $gen) samples = 10000 evals = 10

BenchmarkTools.Trial: 1336 samples with 10 evaluations per sample.
 Range (min … max):  182.908 μs … 82.685 ms  ┊ GC (min … max): 0.00% … 1.11%
 Time  (median):     202.825 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   434.290 μs ±  3.575 ms  ┊ GC (mean ± σ):  9.72% ± 4.40%

       ▆▇██▇▆▁                                                  
  ▂▂▂▃▄███████▇▆▅▄▄▃▄▃▃▂▃▃▂▂▂▂▂▁▂▂▁▁▁▁▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂ ▃
  183 μs          Histogram: frequency by time          314 μs <

 Memory estimate: 192.12 KiB, allocs estimate: 6.

In [4]:
# defines and stores relevant variables to ghost creation and area calculation
struct GhostedArea
    resolution::Int
    n_ghosts::Int
    rapidity_step::Float64
    phi_step::Float64
    ghost_pt::Float64

    # constructor
    # default value of ghost_pt set to 1.0e-45 as -45 is the smallest magnitude that doesn't result in the value defaulting to 0
    function GhostedArea(resolution::Int)
        new(resolution, 
            resolution * resolution,
            Float64(6.28 / resolution),
            Float64(10.0 / resolution),
            1.0f-45)
    end
end

# sets the pt of ghost particles if default value is not satisfactory
# any number with magnitude <-45 will default to 0 due to calculations done in the PseudoJet constructor
function set_ghost_pt!(desired_pt::Float64, gen::GhostedArea)
    gen.ghost_pt = desired_pt
end

# generates and adds ghosts to an event (a set of PseudoJets)
function add_ghosts!(event::Vector{PseudoJet}, gen::GhostedArea)
    # Set aside memory for output
    n_original = length(event)
    resize!(event, n_original + gen.n_ghosts)
    
    # Generate random values
    rand_vals = rand(Float64, 2 * gen.n_ghosts) .- 0.5f0
    
    # loop without array bounds checking in order to maximize time efficiency
    @inbounds for k in 0:(gen.n_ghosts - 1)
        i = k ÷ gen.resolution
        # modular division means j cycles through 0-99 100 times (acts like an inner loop)
        j = k % gen.resolution
        # index to iterate through the random values
        index = 2k + 1
        
        rap = (i + rand_vals[index]) * gen.rapidity_step
        phi = (j + rand_vals[index + 1]) * gen.phi_step
        
        # k + 1 is due to indexing beginning at 1 in julia
        event[n_original + k + 1] = PseudoJet(pt = gen.ghost_pt, rap = rap, phi = phi)
    end
    
    return event
end

add_ghosts! (generic function with 1 method)

In [5]:
event = Vector{PseudoJet}()
jet1 = PseudoJet(pt = 3, rap = 4, phi = 5, m = 6)
push!(event, jet1)

gen = GhostedArea(100)
add_ghosts!(event, gen)

10001-element Vector{PseudoJet}:
 PseudoJet(px: 0.8509865563896788 py: -2.8767728239894153 pz: 183.06632985935454 E: 183.1891949001742 cluster_hist_index: 0)
 PseudoJet(px: 1.4011387188676278e-45 py: 2.1158369716810633e-47 pz: 1.51613702968595e-47 E: 1.4013804812642317e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.399494515161843e-45 py: 7.108085643131121e-47 pz: -2.344736734818647e-47 E: 1.4014946183109842e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.3825693651726233e-45 py: 2.283403963499667e-46 pz: -2.8322867726734773e-47 E: 1.4015846642123896e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.347944885943696e-45 py: 3.829908230980567e-46 pz: 2.6825334095121855e-47 E: 1.4015552021481015e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.3048565003204208e-45 py: 5.1086876953933504e-46 pz: -1.0388454392864409e-47 E: 1.4013369709330314e-45 cluster_hist_index: 0)
 PseudoJet(px: 1.2539459942976371e-45 py: 6.2550525937357294e-46 pz: -4.112207681821477e-48 E: 1.401304498091371e-45 cluster_hist_index: 0)
 P

In [6]:
@benchmark add_ghosts!($event, $gen) samples = 10000 evals = 10

BenchmarkTools.Trial: 1336 samples with 10 evaluations per sample.
 Range (min … max):  173.012 μs … 113.245 ms  ┊ GC (min … max):  0.00% … 21.67%
 Time  (median):     188.548 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   438.918 μs ±   3.838 ms  ┊ GC (mean ± σ):  15.26% ±  8.07%

  █                                                              
  █▆▅▃▁▁▁▁▁▁▄▁▃▃▁▁▃▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▃ ▇
  173 μs        Histogram: log(frequency) by time        2.4 ms <

 Memory estimate: 320.12 KiB, allocs estimate: 6.